<a href="https://colab.research.google.com/github/phelipe-sb/Rain-Prediction/blob/main/ML%20Model%20Pipeline/Make_Prediction_on_New_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import requests
import time
from datetime import datetime, timedelta
import pandas as pd

In [2]:
%cd /content/drive/MyDrive/Weather_Project

/content/drive/MyDrive/Weather_Project


In [3]:
# Upgrade: Query in BigQuery instead Api

f = open("project_info.txt", "r")
info_list = eval(f.read())

for x in info_list[1]:
    table_id = x
    
for x in info_list[2]:
    path = x
    
for x in info_list[3]:
    url = x
    
headers = info_list[0]

cities = list(("Sao Paulo", "Carapicuiba", "New York", "Paris", "London", "Rome", "Moscow", "Ottawa", "Hong Kong", "Beijing"))
listdict = list(())
for i in cities:
    querystring = {"q":"{}".format(i)}
    response = requests.request("GET", url, headers=headers, params=querystring)
    var = response.json()
    dic = var['current']
    dic['condition'] = dic['condition']['text']
    dic['city'] = "{}".format(i)
    listdict.append(dic)
    time.sleep(1)

In [4]:
df = pd.DataFrame(listdict)
cidades = ['Paris', 'Sao Paulo', 'Carapicuiba', 'New York', 'Otawwa', 'London', 'Rome', 'Moscow'
        , 'Hong Kong', 'Beijing']
df = df.loc[df['city'].isin(cidades)]

In [5]:
# This conditions means that it's raining in the moment
raining_list = ['Light rain', 'Light rain shower', 'Light drizzle', 'Heavy rain', 'Moderate rain', 'Patchy light rain with thunder'
            , 'Moderate or heavy rain shower', 'Patchy light rain', 'Torrential rain shower', 'Moderate rain at times'
            , 'Moderate or heavy rain with thunder']

# Creating the target variable
df['target'] = df['condition'].apply(lambda x: 1 if x in raining_list else 0)
df['target'].unique()

array([0, 1])

In [6]:
x = df[['city', 'temp_c', 'is_day', 'condition', 'wind_kph', 'wind_degree'
          , 'pressure_mb', 'precip_mm', 'humidity', 'cloud', 'feelslike_c', 'vis_km', 'uv'
          , 'gust_kph', 'target']].copy()

x = pd.get_dummies(x, columns=['condition', 'city'])
x.head()

,temp_c,is_day,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feelslike_c,vis_km,uv,gust_kph,target,condition_Clear,condition_Mist,condition_Moderate rain,condition_Partly cloudy,condition_Sunny,city_Beijing,city_Carapicuiba,city_Hong Kong,city_London,city_Moscow,city_New York,city_Paris,city_Rome,city_Sao Paulo
0,22.0,0,6.8,100,1016.0,0.0,41,0,23.7,10.0,1.0,14.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,22.0,0,6.8,100,1016.0,0.0,41,0,24.1,10.0,1.0,22.7,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,18.9,0,0.0,323,1013.0,1.0,68,0,18.9,16.0,1.0,31.3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,14.0,0,22.0,220,1016.0,0.0,82,75,12.6,10.0,1.0,25.9,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,12.0,0,16.9,250,1012.0,6.9,88,25,10.4,10.0,1.0,22.7,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [7]:
df_mask = pickle.load(open('df_mask', 'rb'))
x = df_mask.append(x)
x = x.fillna(0)

In [8]:
model = pickle.load(open('model_v1', 'rb'))
pred = model.predict(x)
results = model.predict_proba(x)

In [9]:
prob_list = []
for p in results:
  prob_list.append(1 - p[0])

In [11]:
df['prob_chuva'] = pd.DataFrame(prob_list)
df['pred'] = pd.DataFrame(pred)
df['last_updated'] = df['last_updated'].astype('datetime64[ns]')
df['prediction_time'] = df['last_updated'] + pd.Timedelta(hours=4)
df = df[['prediction_time', 'city', 'prob_chuva', 'pred']]
df

,prediction_time,city,prob_chuva,pred
0,2021-09-29 03:45:00,Sao Paulo,0.003128,0.0
1,2021-09-29 03:45:00,Carapicuiba,0.003215,0.0
2,2021-09-29 02:45:00,New York,0.011784,0.0
3,2021-09-29 08:45:00,Paris,0.899018,1.0
4,2021-09-29 07:45:00,London,0.065019,0.0
5,2021-09-29 08:45:00,Rome,0.013201,0.0
6,2021-09-29 09:45:00,Moscow,0.035490,0.0
8,2021-09-29 14:45:00,Hong Kong,0.017749,0.0
9,2021-09-29 14:45:00,Beijing,NaN,NaN
